## Inverse Cooking: Recipe Generation from Food Images

In [12]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from args import get_parser
import pickle
from model import get_model
from torchvision import transforms
from utils.output_utils import prepare_output
from PIL import Image
import time

class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word, idx=None):
        if idx is None:
            if not word in self.word2idx:
                self.word2idx[word] = self.idx
                self.idx2word[self.idx] = word
                self.idx += 1
            return self.idx
        else:
            if not word in self.word2idx:
                self.word2idx[word] = idx
                if idx in self.idx2word.keys():
                    self.idx2word[idx].append(word)
                else:
                    self.idx2word[idx] = [word]

                return idx

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<pad>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)
    
def get_true_ingre(ingrs_vocab, labels):

    ilabels_gt = np.ones(20) * ingrs_vocab('<pad>')
    pos = 0

    true_ingr_idxs = []
    for i in range(len(labels)):
        true_ingr_idxs.append(ingrs_vocab(labels[i]))

    for i in range(20):
        if i >= len(labels):
            label = '<pad>'
        else:
            label = labels[i]
        label_idx = ingrs_vocab(label)
        if label_idx not in ilabels_gt:
            ilabels_gt[pos] = label_idx
            pos += 1

    ilabels_gt[pos] = ingrs_vocab('<end>')
    ingrs_gt = torch.from_numpy(ilabels_gt).long()
    true_ingrs = ingrs_gt
    return true_ingrs


def mask_from_eos(ids, eos_value, mult_before=True):
    mask = torch.ones(ids.size()).to(device).byte()
    mask_aux = torch.ones(ids.size(0)).to(device).byte()

    # find eos in ingredient prediction
    for idx in range(ids.size(1)):
        # force mask to have 1s in the first position to avoid division by 0 when predictions start with eos
        if idx == 0:
            continue
        if mult_before:
            mask[:, idx] = mask[:, idx] * mask_aux
            mask_aux = mask_aux * (ids[:, idx] != eos_value)
        else:
            mask_aux = mask_aux * (ids[:, idx] != eos_value)
            mask[:, idx] = mask[:, idx] * mask_aux
    return mask

Set ```data_dir``` to the path including vocabularies and model checkpoint

In [13]:
data_dir = ''

In [14]:
# code will run in gpu if available and if the flag is set to True, else it will run on cpu
use_gpu = True
device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
map_loc = None if torch.cuda.is_available() and use_gpu else 'cpu'


# device = torch.device('cpu')
# map_loc = None


In [61]:
print(device)

cuda


In [15]:
# code below was used to save vocab files so that they can be loaded without Vocabulary class
#ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_ingrs.pkl'), 'rb'))
#ingrs_vocab = [min(w, key=len) if not isinstance(w, str) else w for w in ingrs_vocab.idx2word.values()]
#vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_toks.pkl'), 'rb')).idx2word
#pickle.dump(ingrs_vocab, open('../demo/ingr_vocab.pkl', 'wb'))
#pickle.dump(vocab, open('../demo/instr_vocab.pkl', 'wb'))

ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'ingr_vocab.pkl'), 'rb'))
vocab = pickle.load(open(os.path.join(data_dir, 'instr_vocab.pkl'), 'rb'))

ingr_vocab_size = len(ingrs_vocab)
instrs_vocab_size = len(vocab)
output_dim = instrs_vocab_size

In [16]:
t = time.time()
import sys; sys.argv=['']; del sys
args = get_parser()
args.maxseqlen = 15
args.ingrs_only=False
model = get_model(args, ingr_vocab_size, instrs_vocab_size)
# Load the trained model parameters
model_path = os.path.join('/home/ct2020dl5787/inversecooking/model/checkpoints', 'modelbest.ckpt')
model.load_state_dict(torch.load(model_path, map_location=map_loc))
model.to(device)
model.eval()
model.ingrs_only = False
model.recipe_only = False
model.reduction = 'none'
print ('loaded model')
print ("Elapsed time:", time.time() -t)


/home/ct2020dl5787/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


loaded model
Elapsed time: 2.5731148719787598


In [17]:
transf_list_batch = []
transf_list_batch.append(transforms.ToTensor())
transf_list_batch.append(transforms.Normalize((0.485, 0.456, 0.406), 
                                              (0.229, 0.224, 0.225)))
to_input_transf = transforms.Compose(transf_list_batch)

In [18]:
greedy = [True, False, False, False]
beam = [-1, -1, -1, -1]
temperature = 1.0
numgens = len(greedy)

Set ```use_urls = True``` to get recipes for images in ```demo_urls```. 

You can also set ```use_urls = False``` and get recipes for images in the path in ```data_dir/test_imgs```.

In [43]:
import requests
from io import BytesIO
import random
from collections import Counter
unpickled_df = pd.read_pickle("df_recipe1023.pkl")
use_urls = True # set to true to load images from demo_urls instead of those in test_imgs folder
show_anyways = False #if True, it will show the recipe even if it's not valid
image_folder = os.path.join('/home/ct2020dl5787/inversecooking/data/'+'demo_imgs')
true_ingre_1k = pd.read_pickle("true_ingre_1k.pickle")
ablation_vocab = pickle.load(open("../data/recipe1m_vocab_ingrs.pkl", "rb" ) )
#TODO1:load data
# if not use_urls:
#     demo_imgs = os.listdir(image_folder)
#     random.shuffle(demo_imgs)

#demo_urls = ['https://food.fnr.sndimg.com/content/dam/images/food/fullset/2013/12/9/0/FNK_Cheesecake_s4x3.jpg.rend.hgtvcom.826.620.suffix/1387411272847.jpeg',
#          'https://www.196flavors.com/wp-content/uploads/2014/10/california-roll-3-FP.jpg']
demo_urls = unpickled_df["url"].tolist()
demo_files = demo_urls if use_urls else demo_imgs




In [22]:
demo_files

['http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/47/91/49/picaYYmb9.jpg']

In [26]:
type(img_file)

str

In [45]:
info_all = []
err_pic = []
start = time.time()
index = 0
for img_file in demo_files:
    print(index)
    index += 1

    
#    if use_urls:
    response = requests.get(img_file)
    try:
        image = Image.open(BytesIO(response.content))
    
            
#     else:
#         image_path = os.path.join(image_folder, img_file)
#         image = Image.open(image_path).convert('RGB')

        transf_list = []
        transf_list.append(transforms.Resize(256))
        transf_list.append(transforms.CenterCrop(224))
        transform = transforms.Compose(transf_list)

        image_transf = transform(image)
        image_tensor = to_input_transf(image_transf).unsqueeze(0).to(device)

       # plt.imshow(image_transf)
       # plt.axis('off')
      #  plt.show()
      #  plt.close()

        num_valid = 1

        generate_ing = []
        generate_recipt = []
        generate_title = []
        generate_score = []
        
        labels = true_ingre_1k[img_file]
        true_ingre = get_true_ingre(ablation_vocab, labels).reshape(-1,20).cuda()
        for i in range(numgens):
           
            with torch.no_grad():
                outputs = model.sample(image_tensor, greedy=greedy[i],temperature=temperature, beam=beam[i], true_ingrs=true_ingre)
           
            ingr_ids = outputs['ingr_ids'].cpu().numpy()
           
            recipe_ids = outputs['recipe_ids'].cpu().numpy()
           
            outs, valid = prepare_output(recipe_ids[0], ingr_ids[0], ingrs_vocab, vocab)
            #TODO2: Save output  
            generate_ing.append(outs['ingrs'])
            generate_recipt.append(outs['recipe'])
            generate_title.append(outs['title'])
            generate_score.append([valid['is_valid'],valid['score']])

        info_all.append([img_file,generate_ing, generate_recipt,generate_title,generate_score])
    except:
        pass
        
        
print(start-time.time())
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [47]:
info_all

[['http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/47/91/49/picaYYmb9.jpg',
  [['cheese', 'onion', 'pepper', 'potato', 'milk', 'salt', 'butter', 'cream'],
   ['cheese', 'onion', 'pepper', 'potato', 'milk', 'salt', 'butter', 'cream'],
   ['cheese', 'onion', 'pepper', 'potato', 'milk', 'salt', 'butter', 'cream'],
   ['cheese', 'onion', 'pepper', 'potato', 'milk', 'salt', 'butter', 'cream']],
  [['Preheat oven to 350 degrees f (175 degrees c).',
    'Grease a 9x13 inch baking dish.',
    'In a large saucepan over medium heat, melt butter.',
    'Stir in onion and cook until tender.',
    'Stir in water, milk, and corn.',
    'Bring to a boil, then reduce heat and simmer for 5 minutes.',
    'Stir in potatoes and simmer for 10 minutes.',
    'Pour into prepared baking dish.',
    'Bake in preheated oven for 30 minutes, or until potatoes are tender.'],
   ['Preheat oven to 350 degrees.',
    'Cook and drain potatoes.',
    'Mix soup, milk, water 

In [46]:
generate_data = pd.DataFrame(info_all,columns = ['url','generate_ingre','generate_reci','generate_title','generate_score'])
all_info = pd.merge(unpickled_df, generate_data , how='inner', on=['url'])
all_info.to_pickle('ablation_1k.pkl')



In [ ]:
#TODO3: Evaluation

In [ ]:
generate_data.to_pickle('output/generate_1000_backup.pkl')

[['Preheat oven to 350 degrees f (175 degrees c).',
  'Grease a 9x13 inch baking dish.',
  'In a large saucepan over medium heat, melt butter.',
  'Stir in onion and cook until tender.',
  'Stir in water, milk, and corn.',
  'Bring to a boil, then reduce heat and simmer for 5 minutes.',
  'Stir in potatoes and simmer for 10 minutes.',
  'Pour into prepared baking dish.',
  'Bake in preheated oven for 30 minutes, or until potatoes are tender.'],
 ['In a heavy saucepan combine the potatoes, water, and onion.',
  'Bring to a boil; reduce heat, cover, and simmer for 20 minutes or until tender.',
  'Mash potatoes with a potato masher, or mash with a potato masher.',
  'Preheat oven to 350 degrees f (175 degrees c).',
  'Stir in corn, butter and milk, then fold in cheese.',
  'Pour soup into a 2-quart baking dish.',
  'Sprinkle with cornflakes.',
  'Bake in preheated oven 35 to 40 minutes, or until the edges are lightly browned.'],
 ['Peel and slice potatoes',
  'Boil in water with the onion

In [ ]:
unpickled_df.iloc[0]["ingredients"]

In [36]:
pickle.load(open('ablation_1k.pkl', 'rb'))

,id,ingredients,instructions,url,generate_ingre,generate_reci,generate_title,generate_score
0,00003a70b1,"[2 12 cups milk, 1 12 cups water, 14 cup butte...","[Preheat oven to 350 degrees Fahrenheit., Spra...","http://img.sndimg.com/food/image/upload/w_512,...","[[cheese, onion, pepper, potato, milk, salt, b...",[[Preheat oven to 350 degrees f (175 degrees c...,"[Scalloped potatoes, Baked potato soup, Potato...","[[True, 0.5392156862745098], [True, 0.57142857..."
1,000075604a,"[2 Chicken thighs, 2 tsp Kombu tea, 1 White pe...",[Pierce the skin of the chicken with a fork or...,https://img-global.cpcdn.com/001_recipes/58069...,"[[chicken, pepper, salt, lemon, oil, paprika, ...",[[Preheat oven to 350 degrees f (175 degrees c...,"[Chicken drums - easy, Chicken in the crock po...","[[True, 0.6911764705882353], [True, 0.60377358..."
2,00007bfd16,"[6 -8 cups fresh rhubarb, or, 6 -8 cups frozen...",[Put ingredients in a buttered 9 x 12 x 2-inch...,"http://img.sndimg.com/food/image/upload/w_512,...","[[sugar, egg, flour, rhubarb, salt, butter, ci...",[[Preheat oven to 350 degrees f (175 degrees c...,"[Rhubarb dump cake, Rhubarb pie, Rhubarb dump ...","[[True, 0.5625], [True, 0.6352941176470588], [..."
3,000095fc1d,"[8 ounces, weight Light Fat Free Vanilla Yogur...",[Layer all ingredients in a serving dish.],http://tastykitchen.com/recipes/wp-content/upl...,"[[strawberries, yogurt, blueberries, banana, a...","[[Layer yogurt, strawberries and vanilla in pa...","[Yogurt parfait, Creamy strawberry parfait, Lo...","[[True, 0.7222222222222222], [True, 0.53703703..."
4,0000b1e2b5,"[1 teaspoon fennel seeds, 1 pound pork tenderl...","[Preheat oven to 350F with rack in middle., Cr...",http://assets.epicurious.com/photos/5609a4d662...,"[[pepper, oil, salt, clove, onion, parsley], [...","[[Preheat oven to 400 degrees f., Season pork ...","[Pork tenderloin with fennel and garlic, Brais...","[[True, 0.44680851063829785], [True, 0.4729729..."


In [50]:
ablation_vocab = pickle.load(open("../data/recipe1m_vocab_ingrs.pkl", "rb" ) )
labels = ["juice", "tomato", "beef"]

In [51]:
true_ingrs = get_true_ingre(ablation_vocab, labels).reshape(-1,20).cuda()

In [52]:
true_ingrs


tensor([[  22,  166,   20,    0, 1487, 1487, 1487, 1487, 1487, 1487, 1487, 1487,
         1487, 1487, 1487, 1487, 1487, 1487, 1487, 1487]], device='cuda:0')

In [ ]:
input_mask = mask_from_eos(true_ingrs, eos_value=0, mult_before=False)

In [ ]:
input_mask